In [ ]:
# default_exp web_scraping

# web scraping

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import datetime
import re
import csv
import os
from fastcore.script import *

def getHTMLsfu(): Funcion que  por medio del driver de selenium hace el log en la EESSA y obtiene el código html conla información

In [ ]:
#export
def getHTMLsfu(currentdate,originaldate):

    source_code=""
    try:
        rutadriver=back()
        rutadriver=str(rutadriver)+"geckodriver"
        #rutadriver=
        options = Options()
        options.add_argument('-headless')
        driver = webdriver.Firefox(executable_path=str(rutadriver), options=options)
        currentday = datetime.datetime.now()
        #driver.get("https://esc.cbk.waw.pl/products/api.php?parameter=f10_7&start_date=2019%2F01%2F01+00%3A00&end_date="+str(currentday.year)+"%2F"+str(currentday.month)+"%2F"+str(currentday.day)+"+00%3A00&output_type=html")
        driver.get("https://esc.cbk.waw.pl/products/api.php?parameter=f10_7&start_date="+str(originaldate.year)+"%2F"+str(originaldate.month)+"%2F"+str(originaldate.day)+"+00%3A00&end_date="+str(currentdate.year)+"%2F"+str(currentdate.month)+"%2F"+str(currentdate.day)+"+00%3A00&output_type=html")
        #driver.get("https://esc.cbk.waw.pl/products/api.php?parameter=f10_7&start_date="+str(currentday.year-differenceyears)+"%2F"+str(currentday.month)+"%2F"+str(currentday.day)+"+00%3A00&end_date="+str(currentday.year)+"%2F"+str(currentday.month)+"%2F"+str(currentday.day)+"+00%3A00&output_type=html")
        #driver.get("https://esc.cbk.waw.pl/products/api.php?parameter=f10_7&start_date=1750%2F01%2F01+00%3A00&end_date="+str(currentday.year)+"%2F"+str(currentday.month)+"%2F"+str(currentday.day)+"+00%3A00&output_type=html")
        sleep(random.uniform(8.0,10.0))

        #getHTML(driver)
        user = driver.find_element_by_name("callback_0")
        password = driver.find_element_by_name("callback_1")
        button = driver.find_element_by_name("callback_2")

        credentials=getCredentials()
        user.send_keys(credentials[0])
        password.send_keys(credentials[1])
        button.click()
        sleep(random.uniform(10.0,12.0))

        elem = driver.find_element_by_xpath("//*")
        source_code = elem.get_attribute("outerHTML")

    finally:
        driver.quit()
    return source_code




back(): Devuelve el path de la carpeta anterior

In [ ]:
#export
def back():
    path=os.getcwd()
    s=path.split('/')
    length=len(s)
    back_path=""
    for pathpart in range(0,length-1):
        if pathpart==0:
            back_path=s[pathpart]+"/"
        else:
            back_path=back_path+s[pathpart]+"/"
    return str(back_path)



getCredentials(): Funcion que obtiene el user y password previamente escrito en el fichero infouser.txt para el login de la EESA

In [ ]:
#export
def getCredentials():
    credentials=[]
    user=getInfo("usuario")
    password=getInfo("password")
    credentials.append(user)
    credentials.append(password)
    return credentials


In [ ]:
#export
def getInfo(element):
    try:
        txtpath=back()
        txtpath=str(txtpath)+"/infouser.txt"
        f = open (txtpath,'r')
        message = f.read()
        info=re.findall(str(element)+"='(.*?)'",message)[0]
    finally:
        f.close()
    return info

getDatosSfu(html): Función que analiza y obtiene los datos (fecha y valor de sfu) del html

In [ ]:
#export
def getListDataSfu(html):
    #print(html)
    dailysfu=re.findall('<tr>\n<td>(.*?)/td></tr>',str(html),re.DOTALL)
    datalist=[]
    for day in dailysfu:
        if(re.findall("(.*?)20:00:00</td>",day)):
            datedata=re.findall("(.*?)20:00:00</td>",day)
            sfudata=re.findall("<td>(.*?)<",day)
            datalist.append([datedata[0],sfudata[0]])
    return datalist

getValuesSfu(html):Obtiene todos los valores de sfu hasta la fecha actual

In [ ]:
#export
def getValuesSfu(html):
    #Hasta 1996-02-13 solo habia una unica toma por dia, se tiene que tratar diferente antes de esa fecha(oldsfu)
    newsfu=re.findall('<td>1996-02-13 00:00:00</td><td>68</td></tr>(.*?)</tbody></table></div></body></html>',str(html),re.DOTALL)
    oldsfu=re.findall('<tbody><tr>(.*?)<td>1996-02-14',str(html),re.DOTALL)
    datalist=[]
    sfudiario=re.findall('20:00:00</td><td>(.*?)</td>',str(newsfu),re.DOTALL)
    for value in sfudiario:
        datalist.append(value)
    
    sfudiario=re.findall('00:00:00</td><td>(.*?)</td>',str(oldsfu),re.DOTALL)
    for value in sfudiario:
        datalist.append(value)
    return datalist

sfuScv(listaDatos): Función que guarda los datos(fecha y valor de sfu) en un fichero csv

In [ ]:
#export
def sfuScv(datalist):
    currentday = datetime.datetime.now()
    #nombredatos=str(currentday.year)+"-"+str(currentday.month)+"-"+str(currentday.day)+"sfuData.csv"
    datanames=str(currentday.year)+"-"+str(currentday.month)+"-"+str(currentday.day)+"-"+str(currentday.hour)+"sfuData.csv"
    path=getInfo("csvdirectory")
    try:
        myFile = open(str(path)+str(datanames), 'w',newline='')
        with myFile:
                writer = csv.writer(myFile)
                writer.writerows(datalist)

    finally:
        myFile.close()



In [ ]:
#export
def web_scraping():
    datosSfu=[]
    
    currentdate = datetime.datetime.now()
    originaldate=datetime.datetime(year=currentdate.year-1, month=currentdate.month, day=currentdate.day)
    sourcecode=getHTMLsfu(currentdate,originaldate)
    
    datosSfu=getListDataSfu(sourcecode)
    
    sfuScv(datosSfu)


getMaxValueSfu(): Saca el mayor valor de sfu historicamente

In [ ]:
#export
def getMaxValueSfu():
    sfuValues=[]
    currentdate = datetime.datetime.now()
    originaldate=datetime.datetime(1949, 1, 1)
    sourcecode=getHTMLsfu(currentdate,originaldate)
    sfuValues=getValuesSfu(sourcecode)
    return max(sfuValues, key=float)


In [ ]:
#hide
getMaxValueSfu()

'938.6'

In [ ]:
#export
@call_parse
def main():
    web_scraping()


In [ ]:
"""
18:00 = 592.4
20:00 = 938.6
22:00 = 383 // 331
00:00 = 383 
"""